In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

import networkx as nx
from scipy import sparse

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, GATConv, GINConv, global_max_pool, GlobalAttention, GatedGraphConv
from torch_geometric.data import Data, DataLoader
from torch_geometric.utils import softmax
from torch_geometric.utils.convert import from_scipy_sparse_matrix


from model import GCNNet
from preprocess import build_graph
from train import train

In [2]:
M = 2
F = 3

#M: total number of orbitals
#N: total number of particles
#F: feature vector length

#A is potential matrix: M x M
#U is coulumb 4-tensor: M x M x M x M
#X is additional orbital feature matrix: M x F
#Y is additional pairwise orbital feature matrix: M x M x F

#E is ground state energy

data_list = []

for i in range(4):
    A = np.random.normal(size = (M, M))
    U = np.random.normal(size = (M, M, M, M))
    X = np.random.normal(size = (M, F))
    Y = np.random.normal(size = (M, M, F))
    
    E = np.random.random()

    W, Z = build_graph(A, U, X, Y)
    Z = torch.from_numpy(Z)
    edge_index, edge_attr = from_scipy_sparse_matrix(sparse.coo_matrix(W))    
    data = Data(x = Z, edge_index = edge_index, edge_attr = edge_attr, y = E)
    data_list.append(data)

loader = DataLoader(data_list, batch_size = 2)

In [4]:
model = GCNNet(input_dim = F, hidden_dim = 30).double()
losses = train(model, loader, iterations = 100)
print(losses[::10])

[0.7360441742005637, 0.059490672328416073, 0.00447561669870963, 0.0019456016415455865, 4.339877145534711e-06, 0.00012032453872655869, 8.86710555931685e-05, 7.903137841783182e-05, 7.063503954721852e-05, 1.720161186137826e-05, 1.722462833661854e-09, 1.844370158447662e-06, 7.420332971089333e-07, 5.146963044286444e-09, 7.792297246950968e-08, 1.5575514296511242e-08, 4.927389575884218e-09, 3.88690576838448e-09, 4.1968665599806415e-10, 6.058246503860758e-10]
